# IAM Handwriting Dataset Preparation for CycleGAN

This notebook processes the IAM handwriting forms dataset to prepare it for CycleGAN training:
1. Extract writer IDs from image filenames
2. Split writers into two groups (Domain A and Domain B) for style transfer
3. Create train/test splits while preventing data leakage
4. Organize images into CycleGAN-compatible folder structure: `trainA/`, `trainB/`, `testA/`, `testB/`

**CycleGAN learns to translate between two domains (e.g., Writer Group A ↔ Writer Group B handwriting styles)**

## Step 1: Install Required Libraries

Install all necessary Python packages

In [ ]:
# Install required libraries
!pip install pandas scikit-learn numpy

## Step 2: Import Libraries

In [ ]:
import os
import pandas as pd
import shutil
from pathlib import Path
from sklearn.model_selection import train_test_split
import random
from collections import defaultdict

# Set random seed for reproducibility
random.seed(42)

## Step 3: Define Dataset Path and Configuration

In [ ]:
# Define base directory containing the IAM dataset
base_dir = "/home/studentiotlab/image_to_image/data/iam_forms_dataset/data"

# Output directory for CycleGAN-formatted dataset
output_dir = "/home/studentiotlab/image_to_image/data/iam_cyclegan"

# Split ratios
TRAIN_RATIO = 0.85  # 85% for training
TEST_RATIO = 0.15   # 15% for testing

# Domain split ratio (how to divide writers into Domain A and B)
DOMAIN_A_RATIO = 0.5  # 50% of writers go to Domain A, 50% to Domain B

print(f"Input directory: {base_dir}")
print(f"Output directory: {output_dir}")
print(f"Split ratios - Train: {TRAIN_RATIO}, Test: {TEST_RATIO}")
print(f"Domain split - Domain A: {DOMAIN_A_RATIO*100}%, Domain B: {(1-DOMAIN_A_RATIO)*100}%")

## Step 4: Traverse Dataset and Extract Writer IDs

Find all .png files and extract writer ID from each filename

In [ ]:
# Lists to store image paths and writer IDs
image_paths = []
writer_ids = []

# Traverse the directory recursively to find all .png files
print("Scanning dataset directory for .png files...")
for root, dirs, files in os.walk(base_dir):
    for filename in files:
        if filename.endswith('.png'):
            # Get full absolute path
            full_path = os.path.abspath(os.path.join(root, filename))
            
            # Extract writer ID (part before the first '-')
            # Example: a01-000u.png -> a01
            writer_id = filename.split('-')[0]
            
            image_paths.append(full_path)
            writer_ids.append(writer_id)

print(f"Found {len(image_paths)} images")
print(f"Example filenames and writer IDs:")
for i in range(min(5, len(image_paths))):
    print(f"  {os.path.basename(image_paths[i])} -> Writer ID: {writer_ids[i]}")

## Step 5: Create Writer Labels CSV

In [ ]:
# Create DataFrame
df = pd.DataFrame({
    'image_path': image_paths,
    'writer_id': writer_ids
})

# Save to CSV in the output directory
csv_path = os.path.join(output_dir, 'writer_labels.csv')
os.makedirs(output_dir, exist_ok=True)  # Create output dir if it doesn't exist
df.to_csv(csv_path, index=False)

print(f"CSV file saved to: {csv_path}")
print(f"\nDataFrame shape: {df.shape}")
print(f"\nFirst few entries:")
print(df.head(10))

## Step 6: Split Writers into Domain A and Domain B

For CycleGAN, we need two domains. We'll split writers into two groups to learn style transfer between different handwriting styles.

In [ ]:
# Get unique writer IDs
unique_writers = df['writer_id'].unique()
print(f"Total unique writers: {len(unique_writers)}")

# First split: Divide writers into Domain A and Domain B (for CycleGAN)
domain_A_writers, domain_B_writers = train_test_split(
    unique_writers, 
    test_size=(1 - DOMAIN_A_RATIO), 
    random_state=42
)

print(f"\nDomain distribution:")
print(f"  Domain A writers: {len(domain_A_writers)}")
print(f"  Domain B writers: {len(domain_B_writers)}")

# Second split: For each domain, split writers into train/test
trainA_writers, testA_writers = train_test_split(
    domain_A_writers,
    test_size=TEST_RATIO,
    random_state=42
)

trainB_writers, testB_writers = train_test_split(
    domain_B_writers,
    test_size=TEST_RATIO,
    random_state=42
)

print(f"\nWriter distribution per split:")
print(f"  Domain A - Train: {len(trainA_writers)}, Test: {len(testA_writers)}")
print(f"  Domain B - Train: {len(trainB_writers)}, Test: {len(testB_writers)}")

# Create sets for quick lookup
trainA_writer_set = set(trainA_writers)
testA_writer_set = set(testA_writers)
trainB_writer_set = set(trainB_writers)
testB_writer_set = set(testB_writers)

# Assign each image to its corresponding split
def assign_cyclegan_split(writer_id):
    if writer_id in trainA_writer_set:
        return 'trainA'
    elif writer_id in testA_writer_set:
        return 'testA'
    elif writer_id in trainB_writer_set:
        return 'trainB'
    elif writer_id in testB_writer_set:
        return 'testB'
    else:
        return 'unknown'

df['split'] = df['writer_id'].apply(assign_cyclegan_split)

# Count images per split
split_counts = df['split'].value_counts()
print(f"\nImage distribution:")
for split_name in ['trainA', 'testA', 'trainB', 'testB']:
    count = split_counts.get(split_name, 0)
    percentage = (count / len(df)) * 100
    print(f"  {split_name}: {count} images ({percentage:.1f}%)")

## Step 7: Create CycleGAN Directory Structure

Create the folder structure that CycleGAN expects: `trainA/`, `trainB/`, `testA/`, `testB/`

In [ ]:
# Create CycleGAN directory structure
trainA_dir = os.path.join(output_dir, 'trainA')
testA_dir = os.path.join(output_dir, 'testA')
trainB_dir = os.path.join(output_dir, 'trainB')
testB_dir = os.path.join(output_dir, 'testB')

# Create directories if they don't exist
os.makedirs(trainA_dir, exist_ok=True)
os.makedirs(testA_dir, exist_ok=True)
os.makedirs(trainB_dir, exist_ok=True)
os.makedirs(testB_dir, exist_ok=True)

print(f"Created CycleGAN directory structure:")
print(f"  trainA: {trainA_dir}")
print(f"  testA: {testA_dir}")
print(f"  trainB: {trainB_dir}")
print(f"  testB: {testB_dir}")

## Step 8: Copy Images to CycleGAN Folders

Copy each image to its corresponding domain and split directory

In [ ]:
# Copy images to their respective split directories
print("Copying images to CycleGAN directory structure...")
copy_count = {'trainA': 0, 'testA': 0, 'trainB': 0, 'testB': 0}

for idx, row in df.iterrows():
    src_path = row['image_path']
    split = row['split']
    
    # Determine destination directory
    if split == 'trainA':
        dst_dir = trainA_dir
    elif split == 'testA':
        dst_dir = testA_dir
    elif split == 'trainB':
        dst_dir = trainB_dir
    elif split == 'testB':
        dst_dir = testB_dir
    else:
        continue
    
    # Get filename and create destination path
    filename = os.path.basename(src_path)
    dst_path = os.path.join(dst_dir, filename)
    
    # Copy the file
    try:
        shutil.copy2(src_path, dst_path)
        copy_count[split] += 1
        
        # Print progress every 100 images
        if (idx + 1) % 100 == 0:
            print(f"  Processed {idx + 1}/{len(df)} images...")
    except Exception as e:
        print(f"Error copying {src_path}: {e}")

print(f"\nCopying complete!")
print(f"Files copied:")
print(f"  trainA: {copy_count['trainA']}")
print(f"  testA: {copy_count['testA']}")
print(f"  trainB: {copy_count['trainB']}")
print(f"  testB: {copy_count['testB']}")

## Step 9: Display Summary Statistics

In [ ]:
print("="*80)
print("CYCLEGAN DATASET PREPARATION SUMMARY")
print("="*80)

# Total statistics
print(f"\n📊 Overall Statistics:")
print(f"  Total images: {len(df)}")
print(f"  Total unique writers: {len(unique_writers)}")

# Domain distribution
print(f"\n🎨 Domain Distribution:")
print(f"  Domain A writers: {len(domain_A_writers)} ({len(domain_A_writers)/len(unique_writers)*100:.1f}%)")
print(f"  Domain B writers: {len(domain_B_writers)} ({len(domain_B_writers)/len(unique_writers)*100:.1f}%)")

# Writer distribution per split
print(f"\n👤 Writer Distribution:")
print(f"  trainA: {len(trainA_writers)} writers")
print(f"  testA: {len(testA_writers)} writers")
print(f"  trainB: {len(trainB_writers)} writers")
print(f"  testB: {len(testB_writers)} writers")

# Image distribution
print(f"\n🖼️  Image Distribution:")
for split_name in ['trainA', 'testA', 'trainB', 'testB']:
    count = split_counts.get(split_name, 0)
    percentage = (count / len(df)) * 100
    print(f"  {split_name}: {count} images ({percentage:.1f}%)")

# Images per writer statistics
print(f"\n📈 Images per Writer Statistics:")
writer_image_counts = df.groupby('writer_id').size()
print(f"  Mean images per writer: {writer_image_counts.mean():.2f}")
print(f"  Median images per writer: {writer_image_counts.median():.0f}")
print(f"  Min images per writer: {writer_image_counts.min()}")
print(f"  Max images per writer: {writer_image_counts.max()}")

# Data leakage check
print(f"\n✅ Data Leakage Check:")
all_splits = [trainA_writer_set, testA_writer_set, trainB_writer_set, testB_writer_set]
split_names = ['trainA', 'testA', 'trainB', 'testB']

leakage_found = False
for i in range(len(all_splits)):
    for j in range(i+1, len(all_splits)):
        overlap = all_splits[i].intersection(all_splits[j])
        if len(overlap) > 0:
            print(f"  ✗ WARNING: {split_names[i]}-{split_names[j]} overlap: {len(overlap)} writers")
            leakage_found = True

if not leakage_found:
    print("  ✓ No data leakage detected - all writers are in exactly one split!")

# Sample entries
print(f"\n📄 Sample Entries:")
sample_df = df[['image_path', 'writer_id', 'split']].groupby('split').head(2)
for split in ['trainA', 'testA', 'trainB', 'testB']:
    split_samples = df[df['split'] == split].head(2)
    if len(split_samples) > 0:
        print(f"\n  {split}:")
        for _, row in split_samples.iterrows():
            print(f"    {os.path.basename(row['image_path'])} (writer: {row['writer_id']})")

print(f"\n📁 Output Files:")
print(f"  CSV file: {csv_path}")
print(f"  CycleGAN dataset: {output_dir}")
print(f"    ├── trainA/ ({copy_count.get('trainA', 0)} images)")
print(f"    ├── testA/ ({copy_count.get('testA', 0)} images)")
print(f"    ├── trainB/ ({copy_count.get('trainB', 0)} images)")
print(f"    └── testB/ ({copy_count.get('testB', 0)} images)")

print(f"\n🚀 Ready for CycleGAN Training!")
print(f"  Use this command to train:")
print(f"  DATA_ROOT={output_dir} name=iam_handwriting_model th train.lua")
print("="*80)